In [30]:
import gymnasium as gym
import numpy as np
import os
import torch
import matplotlib.pyplot as plt
import imageio
import time

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import SubprocVecEnv, VecNormalize, DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback, CallbackList
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

In [31]:
# Numero di ambienti paralleli per il training
NUM_ENVS = 4

# Wrapper personalizzato per applicare la ricompensa modificata
class CustomRewardWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)

    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        
        # Accesso ai dati della simulazione MuJoCo
        torso_angle = self.env.unwrapped.data.qpos[2]  # Angolo del torso
        torso_height = self.env.unwrapped.data.qpos[1]  # Altezza del torso

        # Penalità per ribaltamento meno severa (aggiunta invece di sostituzione)
        if torso_angle < -0.7 and torso_height < 0.3:
            reward += -200  # Sottrai la penalità invece di sostituire la reward
            terminated = True  # Termina l'episodio
        
        return obs, reward, terminated, truncated, info

# Funzione per creare un ambiente monitorato con custom reward
def make_env():
    def _init():
        env = gym.make("HalfCheetah-v5",
                        reset_noise_scale=0.18925327466415615,
                        forward_reward_weight=1.158890288504633,
                        ctrl_cost_weight=0.05108108521573771)
        env = Monitor(env)
        env = CustomRewardWrapper(env)  # Applica il custom reward
        return env
    return _init

# Creazione degli ambienti per il training (con parallelizzazione)
env = SubprocVecEnv([make_env() for _ in range(NUM_ENVS)])
env = VecNormalize(env, norm_obs=True, norm_reward=False, clip_obs=10.)  # Normalizza solo osservazioni

# Selezione automatica del device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Parametri del modello
model_params = {
    "policy": "MlpPolicy",
    "env": env,
    "learning_rate": 8.272618650819588e-05,
    "n_steps": 2048,
    "batch_size": 128,
    "n_epochs": 10,
    "gamma": 0.9808272185952741,
    "gae_lambda": 0.9080997013001573,
    "clip_range": 0.29879721247771235,
    "ent_coef": 0.02650410588466885,
    "verbose": 1,
    "tensorboard_log": "./ppo_HalfCheetah_tensorboard/",
    "device": device,
    "policy_kwargs": dict(net_arch=[256, 256, 128])
}

# Creazione dell'ambiente di valutazione con il custom reward
eval_env = DummyVecEnv([make_env()])
eval_env = VecNormalize(eval_env, norm_obs=True, norm_reward=False, clip_obs=10., training=False)

# Callback per valutazione e salvataggi
eval_callback = EvalCallback(eval_env, best_model_save_path="./logs/best_model",
                             log_path="./logs/", eval_freq=5000, deterministic=True, render=False)
checkpoint_callback = CheckpointCallback(save_freq=5000, save_path="./logs/checkpoints/",
                                         name_prefix="ppo_halfcheetah_checkpoint")

# Creazione e training del modello
model = PPO(**model_params)
model.learn(total_timesteps=1_000_000, callback=CallbackList([eval_callback, checkpoint_callback]))

# Salvataggio del modello e normalizzazione
model.save("ppo_HalfCheetah_model")
env.save("vecnormalize_HalfCheetah.pkl")

# Caricamento del modello e della normalizzazione per la valutazione
model = PPO.load("ppo_HalfCheetah_model", device=device)
eval_env = VecNormalize.load("vecnormalize_HalfCheetah.pkl", eval_env)
eval_env.training = False
eval_env.reset()

# Funzione per la valutazione
def evaluate_agent(model, env, episodes=100):
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=episodes, deterministic=True)
    print(f"Mean Reward: {mean_reward:.2f} ± {std_reward:.2f}")
    return mean_reward, std_reward

# Valutazione del modello allenato
mean_reward_trained, std_reward_trained = evaluate_agent(model, eval_env, episodes=100)


Using cpu device
Logging to ./ppo_HalfCheetah_tensorboard/PPO_8
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -174     |
| time/              |          |
|    fps             | 9377     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 8192     |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1e+03         |
|    ep_rew_mean          | -177          |
| time/                   |               |
|    fps                  | 5310          |
|    iterations           | 2             |
|    time_elapsed         | 3             |
|    total_timesteps      | 16384         |
| train/                  |               |
|    approx_kl            | 0.00094108755 |
|    clip_fraction        | 1.22e-05      |
|    clip_range           | 0.299         |
|    entropy_loss       

In [32]:
#5. Salviamo il modello
model.save("ppo_HalfCheetah_model")
env.save("vecnormalize_HalfCheetah.pkl")    # salviamo anche i parametri di normalizzazione
